# Week Five: Integration and differentiation

To be graded, your notebook must be runnable start to finish. If you can't make an in-notebook test pass, comment it out for to attempt to get partial credit. You should replace the `...` markers with your code. Do not change the names of the pre-defined ALL_CAPS variables and functions. (If you start from scratch, make sure you match the requested function names and requested ALL_CAPS variables). Other that that, you are free to define or make anything you wish. Remember that functions will often be tested with different numbers than the ones you are given.

In [1]:
# EID is your 6+2 UC Electronic ID
EID = '800226177'
NAME = 'Christopher Basham'

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Problem 1: 2D Trapezoidal rule

You can extend the trapezoidal rule to 2D. I'll give you a few hints:

The weight matrix is:
$$
W_{ij} = \begin{matrix}
1 & 2 & 2 & \cdots & 2 & 2 & 1 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
1 & 2 & 2 & \cdots & 2 & 2 & 1
\end{matrix}
$$

If you use the trick I used in the slides instead of explicitly creating this matrix, you would split the sum into 4 parts. However, it's really quite easy to make the matrix above

The "bin" width $h$ is joined by the 2nd dimension bin width of $k$, and then the trapezoidal rule integral value is 

$$
I_\mathrm{tr} = \frac{1}{4} h k \sum_i \sum_j W_{ij} f(x_{i}, y_{j})
$$

Write a function that takes a 2-parameter function 'f' and integrates it with the given parameters (further explanations are built into the docstring of the function). I'm writing a Numpy-style docstring.

See also: <http://mathfaculty.fullerton.edu/mathews/n2003/SimpsonsRule2DMod.html>

In [175]:
def trap_2d(f, xrange, yrange, xN, yN):
    '''Integrate a 2D function with the trapezoidal rule.
    
    This takes a 2D function and integrates it over a 2D grid.
    This uses the Trapezoidal rule in 2D.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take an x and y value (possibly arrays).
    xrange : (float, float)
        The min and max values for x, as an array
    yrange : (float, float)
        The min and max values for y, as an array
    xN : int
        The number of evaluations along x
    yN : int
        The number of evaluations along y

    Returns
    -------
    float
        The total integral
    
    '''
    # If these are *not* arrays, convert them into arrays. Otherwise, leave them alone.
    # (If we used np.array, we'd needlessly make a copy if it aready was an array)
    xrange = np.asarray(xrange)
    yrange = np.asarray(yrange)
    
    # There are N-1 "bins"
    h = (xrange[1] - xrange[0])/(xN-1)
    k = (yrange[1] - yrange[0])/(yN-1)
    
    #Array of x and y values evenly spaced between ranges.
    xi = np.arange(xrange[0],xrange[1]+h,h)
    yj = np.arange(yrange[0],yrange[1]+k,k)

    #Weight Matrix (Creates an (xN x yN) matrix for the N-1 "bins")
    W_ij = np.zeros((xN,yN))
    for i in range(0,xN):
        for j in range(0,yN):
            if (i,j) == (0,0) or (i,j) == (0,yN-1) or (i,j) == (xN-1,0) or (i,j) == (xN-1,yN-1):
                W_ij[i,j] = 1
            elif i == 0 or i == xN-1 or j == 0 or j == yN-1:
                W_ij[i,j] = 2
            else:
                W_ij[i,j] = 4
    print(W_ij)

    #Double Summation
    summ = np.sum(W_ij[i,j] * f(xi[i],yj[j]) for i in range(0,xN) for j in range(0,yN))

    #Final Area
    print(h,k,summ)
    A = 1/4 * h * k * summ

    return(A)

Feel free to add more tests, I'm adding the simplest one I can think of:

In [176]:
def f(x,y):
    return 1

In [177]:
area = trap_2d(f, [0, 10], [0, 10], 10, 10)
print(area)
assert np.isclose(area, 100)

[[1. 2. 2. 2. 2. 2. 2. 2. 2. 1.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [2. 4. 4. 4. 4. 4. 4. 4. 4. 2.]
 [1. 2. 2. 2. 2. 2. 2. 2. 2. 1.]]
1.1111111111111112 1.1111111111111112 324.0
100.00000000000001


# Problem 2: MC integration

Write a function that performs MC integration of an unknown function of N dimensions. You can use `f(*array)` to call the function with the correct number of arguments (it expands to `f(array[0], array[1], ..., array[N])`). Return **two** values; the estimated integral and the estimated error on the integral.

In [129]:
def mc_integrate(f, N, samples, ranges):
    '''
    Integrate an ND function with a set of sampled MC points.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take N
        values (possibly equal length arrays).
    N : int
        The number of Dimensions
    samples : int
        The number of samples to integrate over
    ranges : array[N,2]
        An array with the min and max for each dimension.

    Returns
    -------
    float
        The total integral
    float
        The estimated error = relative error * the total integral.
    '''
    
    ranges = np.asarray(ranges) # Just making sure ranges is an array
    # Hint: You can turn a 1D array, such as arr = array([a, b]) into a 2D array
    # with arr[:,np.newaxis] or arr[:,None] or arr.reshape(2,1) or arr.reshape(-1,1)
    
    # Our definition of ranges matches what you find in Scipy, and is natural to type,
    # but is a but less natural to use, for example:
    widths = ranges[:,1] + ranges[:,0]
    

    #Creates range of values
    ax = []
    for i in range(0,N):
        new = np.random.uniform(ranges[i,0],ranges[i,1],samples)
        ax.append(new)
    ax = np.asarray(ax)
    

    #Uses Mean Value theorem to find total integral (assuming continuous from a -> b)
    I = np.mean(f(*ax))

    #Gets Actual from Scipy to find Error Value
    integ = integrate.nquad(f,[*ranges])
    actual = integ[0]

    #Estimated Error = Relative Error * Total Integral
    e = abs(I-actual)/actual
    e = I * e


    return I,e

In [146]:
def f(x,y,z):
    return x + y + z

In [162]:
integral, error = mc_integrate(f, 3, 100_000, [[0,1],[0,1],[0,1]])
print(integral, error)

1.499556103795208 0.00044376484223148575


#### For comparison, we could use NQuad from Scipy:

In [109]:
from scipy import integrate

In [149]:
integrate.nquad(f, [[0,1],[0,1],[0,1]])

(1.5, 2.7707360439619496e-14)

# Problem 3: Second derivatives

Write a function that calculates:

$$
\frac{d^2 f(t)}{dt^2} \Biggr\rvert_\mathrm{cd}
=
\frac{
\left[
f(t + h) - f(t)
\right] - \left[
f(t) - f(t - h)
\right]
}{h^2}
$$

(The notes in class had a typo, fixed here.) Write a nice docstring as well.

In [163]:
def d2fdt2(f, t, h):
    '''
        Calculates 2nd Derivative of Given Function at a value.
        ------
        f = function to derive
        t = single point 't' to derive at
        h = distance to next point/accuracy of the derivation
        ------
    '''
    forw = f((t+h))-f(t)
    back = f(t)-f((t-h))

    deriv2 = (forw-back)/h**2

    return deriv2

Test your `d2fdt2` function on a function that you know the derivative for.

In [169]:
def f(t):
    return t**3

In [174]:
t = 2
h = .1
derivate2order = d2fdt2(f,t,h)
assert np.isclose(derivate2order,(6*t))
print(derivate2order)

12.000000000000009
